## Economic Dispatch Problem

The Economic Dispatch Problem (EDP) can be mathematically formulated as follows:

**Objective Function:**

Minimize: 

$$
\sum_{i=1}^{n}C_i(P_i)
$$

where:

- $C_i(P_i)$ is the cost function of generator $i$, typically represented as a quadratic function: $C_i(P_i) = a_iP_i^2 + b_iP_i + c_i$
- $P_i$ is the power generated by generator $i$
- $n$ is the total number of generators 

**Subject to:**

1. Power Balance Constraint: 

$$
\sum_{i=1}^{n}P_i = P_D + P_L
$$

   - $P_D$ is the total power demand
   - $P_L$ is the total power loss

2. Generator Capacity Constraints: $P_{min_i} \leq P_i \leq P_{max_i}$ for each $i$
   - $P_{min_i}$ and $P_{max_i}$ are the minimum and maximum limits of the power that generator $i$ can produce

## Solution Methods

There exist several methods to solve the Economic Dispatch problem. Classical methods include the Lambda-Iteration method and Gradient method. However, these methods may struggle with non-convex problems (due to valve-point effects, multi-fuel options, prohibited operating zones, etc.). For such problems, modern heuristic or metaheuristic algorithms are being used. These include Genetic Algorithms, Particle Swarm Optimization, Ant Colony Optimization, etc.

## Importance of Economic Dispatch

Economic Dispatch is crucial for efficient and economical power system operation. It:

1. Ensures that the generation meets the demand
2. Minimizes the total operational cost
3. Helps maintain a reliable and stable power system

The Economic Dispatch problem becomes more complex with the increasing size of the power system, uncertainty of renewable energy sources, and the introduction of smart grids and electric vehicles. As such, it remains an active area of research in power systems engineering.

## Simplified Economic Dispatch Problem

The economic dispatch problem can be simplified for an educational example, with two generators and ignoring power loss.

**Objective Function:**

Minimize: 

$$
C(P_1, P_2) = a_1P_1^2 + b_1P_1 + c_1 + a_2P_2^2 + b_2P_2 + c_2
$$

Where:

- $C(P_1, P_2)$ is the total cost function, which is the sum of the cost functions of generator 1 and generator 2.
- $a_1P_1^2 + b_1P_1 + c_1$ is the cost function of generator 1.
- $a_2P_2^2 + b_2P_2 + c_2$ is the cost function of generator 2.
- $P_1$ and $P_2$ are the power outputs of generator 1 and generator 2, respectively.

**Subject to:**

1. Power Balance Constraint:

$$
P_1 + P_2 = P_D
$$

Where $P_D$ is the total power demand.

2. Generator Capacity Constraints:

- For Generator 1: $P_{min_1} \leq P_1 \leq P_{max_1}$
- For Generator 2: $P_{min_2} \leq P_2 \leq P_{max_2}$

These constraints ensure the power generated by each generator is within its capacity.

## Vector/Matrix Form 

The same problem can be expressed more concisely in vector/matrix form.

**Objective Function:**

Minimize: 

$$
C(P) = \frac{1}{2} P^TAP + B^TP + C
$$

Where:

- $P$ is a $2 \times 1$ vector representing the power outputs of all generators: $P = [P_1, P_2]^T$.
- $A$ is a $2 \times 2$ diagonal matrix where the $i$-th diagonal element is the quadratic cost coefficient $a_i$ of the $i$-th generator.
- $B$ is a $2 \times 1$ vector where the $i$-th element is the linear cost coefficient $b_i$ of the $i$-th generator.
- $C$ is a scalar representing the sum of constant terms $c_i$ of all generators.

**Subject to:**

1. Power Balance Constraint:

$$
1^TP = P_D
$$

Where $1$ is a vector of ones.

2. Generator Capacity Constraints:

$$
P_{min} \leq P \leq P_{max}
$$

Where $P_{min}$ and $P_{max}$ are $2 \times 1$ vectors representing the minimum and maximum capacity of each generator respectively.

## General Economic Dispatch Problem

The economic dispatch problem can be formulated for "k" generators.

**Objective Function:**

Minimize: 

$$
C(P_1, P_2, ..., P_k) = \sum_{i=1}^{k} (a_iP_i^2 + b_iP_i + c_i)
$$

Where:

- $C(P_1, P_2, ..., P_k)$ is the total cost function, which is the sum of the cost functions of all "k" generators.
- $a_iP_i^2 + b_iP_i + c_i$ is the cost function of the $i$-th generator.
- $P_i$ is the power output of the $i$-th generator.

**Subject to:**

1. Power Balance Constraint:

$$
\sum_{i=1}^{k} P_i = P_D
$$

Where $P_D$ is the total power demand.

2. Generator Capacity Constraints:

For each Generator $i$: $P_{min_i} \leq P_i \leq P_{max_i}$

These constraints ensure the power generated by each generator is within its capacity.

## Vector/Matrix Form 

The same problem can be expressed more concisely in vector/matrix form.

**Objective Function:**

Minimize: 

$$
C(P) = \frac{1}{2} P^TAP + B^TP + C
$$

Where:

- $P$ is a $k \times 1$ vector representing the power outputs of all generators: $P = [P_1, P_2, ..., P_k]^T$.
- $A$ is a $k \times k$ diagonal matrix where the $i$-th diagonal element is the quadratic cost coefficient $a_i$ of the $i$-th generator.
- $B$ is a $k \times 1$ vector where the $i$-th element is the linear cost coefficient $b_i$ of the $i$-th generator.
- $C$ is a scalar representing the sum of constant terms $c_i$ of all generators.

**Subject to:**

1. Power Balance Constraint:

$$
1^TP = P_D
$$

Where $1$ is a vector of ones.

2. Generator Capacity Constraints:

$$
P_{min} \leq P \leq P_{max}
$$

Where $P_{min}$ and $P_{max}$ are $k \times 1$ vectors representing the minimum and maximum capacity of each generator respectively.

## Simplified Economic Dispatch Problem

The economic dispatch problem can be further simplified by only keeping the quadratic term of the cost function for "k" generators.

**Objective Function:**

Minimize: 

$$
C(P_1, P_2, ..., P_k) = \sum_{i=1}^{k} a_iP_i^2
$$

Where:

- $C(P_1, P_2, ..., P_k)$ is the total cost function, which is the sum of the cost functions of all "k" generators.
- $a_iP_i^2$ is the simplified cost function of the $i$-th generator. 
- $P_i$ is the power output of the $i$-th generator.

**Subject to:**

1. Power Balance Constraint:

$$
\sum_{i=1}^{k} P_i = P_D
$$

Where $P_D$ is the total power demand.

2. Generator Capacity Constraints:

For each Generator $i$: $P_{min_i} \leq P_i \leq P_{max_i}$

These constraints ensure the power generated by each generator is within its capacity.

## Vector/Matrix Form 

The same problem can be expressed more concisely in vector/matrix form.

**Objective Function:**

Minimize: 

$$
C(P) = \frac{1}{2} P^TAP
$$

Where:

- $P$ is a $k \times 1$ vector representing the power outputs of all generators: $P = [P_1, P_2, ..., P_k]^T$.
- $A$ is a $k \times k$ diagonal matrix where the $i$-th diagonal element is the quadratic cost coefficient $a_i$ of the $i$-th generator.

**Subject to:**

1. Power Balance Constraint:

$$
1^TP = P_D
$$

Where $1$ is a vector of ones.

2. Generator Capacity Constraints:

$$
P_{min} \leq P \leq P_{max}
$$

Where $P_{min}$ and $P_{max}$ are $k \times 1$ vectors representing the minimum and maximum capacity of each generator respectively.

## Example Instance

In [1]:
# Number of generators
k = 3

# Quadratic cost coefficients for each generator
A = [0.5, 0.3, 0.4]

# Minimum and maximum capacity for each generator
P_min = [30, 20, 40]
P_max = [100, 80, 120]

# Total power demand
P_D = 150

150

## Solving the Problem with JuMP

We'll be using the JuMP package in Julia, which provides a high-level interface for mathematical programming, and the Ipopt solver, which is capable of solving nonlinear optimization problems.

1. Install and import the necessary packages


In [3]:
import Pkg
Pkg.add("JuMP")
Pkg.add("Ipopt")

using JuMP, Ipopt

   Resolving package versions...


   Installed CodecBzip2 ─────────── v0.8.1
   Installed SnoopPrecompile ────── v1.0.3


   Installed MutableArithmetics ─── v1.3.3
   Installed BenchmarkTools ─────── v1.3.2


   Installed CommonSubexpressions ─ v0.3.0
   Installed DiffRules ──────────── v1.15.1


   Installed MathOptInterface ───── v1.22.0


...

  ✓ METIS_jll
  ✓ ASL_jll
  ✓ Hwloc_jll
  ✓ OpenBLAS32_jll


  ✓ MUMPS_seq_jll


  ✓ SPRAL_jll


  ✓ Ipopt_jll


  ✓ Ipopt
  8 dependencies successfully precompiled in 15 seconds. 356 already precompiled.


2. **Define the model**: Here, we define model to be a new optimization problem. We specify that we want to use the Ipopt solver.

In [4]:
model = Model(Ipopt.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Ipopt

3. **Define the decision variables**: The decision variables are the power outputs of the generators, represented by the array P. The power output of each generator should be greater than or equal to 0.

In [5]:
@variable(model, P[1:k] >= 0)

3-element Vector{VariableRef}:
 P[1]
 P[2]
 P[3]

1. **Set the objective function**:  We set the objective of our problem, which is to minimize the total cost of power generation. We use the @NLobjective macro because our objective function is nonlinear. The total cost is the sum of the cost of each generator, which is a quadratic function of the power output of the generator.

In [6]:
@NLobjective(model, Min, sum(A[i]*P[i]^2 for i in 1:k))

5. **Set the constraints**: We add constraints to our problem. The first constraint is the _power balance constraint_, which states that the total power generated must equal the total power demand. The second set of constraints are the _generator capacity constraints_. These state that the power output of each generator must be within its minimum and maximum capacity.

In [7]:
# Power balance constraint
@constraint(model, sum(P[i] for i in 1:k) == P_D)

# Generator capacity constraints
for i in 1:k
    @constraint(model, P[i] >= P_min[i])
    @constraint(model, P[i] <= P_max[i])
end

1. **Solve the problem**:  To solve the problem we use the `optimize!`` function in JuMP. This function takes the model we've defined, including the objective function and constraints, and solves it using the specified optimizer, which is _Ipopt_ in our case.

In [8]:
optimize!(model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.13, running with linear solver MUMPS 5.6.1.

Number of nonzeros in equality constraint Jacobian...:        3
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        3
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality c

Once the solver has found an optimal solution, you can retrieve the optimal power outputs with the value function:

In [9]:
value.(P)

3-element Vector{Float64}:
 38.29787234058661
 63.82978723366875
 47.87234042574464

As we incorporate more renewable energy into our power grid, new challenges arise.  We'll balance power from traditional generators and wind farms to meet demand at the lowest cost. 

We'll consider the operating costs and power limits of generators and the variability of wind power. Our goal is to minimize total generation cost while meeting demand and staying within generator limits.

While our model is a simplified version and excludes factors like ramping constraints and wind power uncertainty, it provides a solid introduction to the complexities of integrating renewable energy into power systems.


### Sets

- $I$: Set of generators, indexed by $i$
- $T$: Set of time periods, indexed by $t$
- $W$: Set of wind turbines, indexed by $w$

### Parameters

- $a_i$: Quadratic cost coefficient for generator $i$
- $P_{\text{min},i}$: Minimum power output of generator $i$
- $P_{\text{max},i}$: Maximum power output of generator $i$
- $W_{t}$: Available wind power at time $t$
- $D_{t}$: Power demand at time $t$

### Variables

- $P_{i,t}$: Power output of generator $i$ at time $t$

### Objective Function

The objective is to minimize the total cost of power generation over all periods, which is now a quadratic function of the power output:

$$\min \sum_{i \in I} \sum_{t \in T} a_i P_{i,t}^2$$

### Constraints

**Power balance**: The total power from all generators plus the available wind power must meet the power demand at each time period:

$$\sum_{i \in I} P_{i,t} + W_{t} = D_{t}, \quad \forall t \in T$$

**Generator limits**: The power output of each generator must be within its capacity limits:

$$P_{\text{min},i} \leq P_{i,t} \leq P_{\text{max},i}, \quad \forall i \in I, \forall t \in T$$

Let's denote the following:

- $a = [a_1, a_2, ..., a_n]$ as the vector of quadratic cost coefficients.
- $P = [P_{1,1}, P_{1,2}, ..., P_{n,t}]$ as the matrix of power outputs, where $n$ is the number of generators and $t$ is the number of time periods.
- $W = [W_1, W_2, ..., W_t]$ as the vector of wind powers.
- $D = [D_1, D_2, ..., D_t]$ as the vector of demands.
- $P_{\text{min}} = [P_{\text{min},1}, P_{\text{min},2}, ..., P_{\text{min},n}]$ and $P_{\text{max}} = [P_{\text{max},1}, P_{\text{max},2}, ..., P_{\text{max},n}]$ as the vectors of minimum and maximum generator outputs.
- $1_{n}$ as a vector of ones of size $n$ (the number of generators).
- $1_{t}$ as a vector of ones of size $t$ (the number of time periods).

### Objective Function

The objective function can be written as:

$$\min \sum_{t \in T} a^T \cdot P_{t}^2$$

This means element-wise squaring of the elements of each column of $P$ (representing a time period), and dot product with $a$.

### Constraints

**Power balance**:

The power balance constraint can be written as:

$$P \cdot \mathbf{1_{n}} + W = D$$

Here, $\mathbf{1_{n}}$ is a column vector of ones of size $n$ (the number of generators), and the dot product $P \cdot \mathbf{1_{n}}$ sums the power output of all generators at each time period, resulting in a vector of size $t$.

**Generator limits**:

The generator limits can be written as:

$$P_{\text{min}} \leq P_t \leq P_{\text{max}}, \quad \forall t \in T$$

That is, for every time period $t$, each generator's power output should be between its minimum and maximum output limit.

In [12]:
#

# Number of generators
n = 3

# Number of time periods
t = 24

# Quadratic cost coefficients for each generator
a = [0.5, 0.4, 0.6]

# Wind powers for each time period
W = [1, 1.2, 1.4, 1.6, 1.8, 2, 2.2, 2.4, 2.6, 2.8, 3, 3.2, 3.4, 3.6, 3.8, 4, 4.2, 4.4, 4.6, 4.8, 5, 5.2, 5.4, 5.6]


# Demand for each time period (representing a 24-hour day)
D = [10, 10, 10, 10, 15, 20, 25, 30, 35, 40, 40, 40, 40, 35, 35, 35, 30, 30, 30, 25, 20, 15, 10, 10]

# Minimum and maximum generator outputs
P_min = [1.0, 2.0, 1.5]
P_max = [24.0, 24.0, 24.0]

3-element Vector{Float64}:
 24.0
 24.0
 24.0

In [23]:
import Pkg; Pkg.add("OSQP")

using JuMP, Ipopt, OSQP
using DataFrames



# Define the model
model = Model(OSQP.Optimizer)

@variable(model, P_min[i] <= P[i = 1:n, t = 1:t] <= P_max[i])

@objective(model, Min, sum(a[i] * P[i,t]^2 for i in 1:n, t in 1:t))

@constraint(model, [t = 1:t], sum(P[i,t] for i in 1:n) + W[t] == D[t])

# Solve the model
optimize!(model)


# Solve the model
optimize!(model)

# Initialize a DataFrame with appropriate columns
output = DataFrame(Generator = Int[], Time = Int[], Power_Output = Float64[])

# Populate the DataFrame with the optimal power outputs
for i in 1:n
    for t in 1:t
        push!(output, [i, t, value(P[i,t])])
    end
end

# Return the DataFrame as output
output

   Resolving package versions...


  No Changes to `~/.julia/environments/v1.9/Project.toml`


  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


-----------------------------------------------------------------
           OSQP v0.6.2  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2021
-----------------------------------------------------------------
problem:  variables n = 72, constraints m = 168
          nnz(P) + nnz(A) = 288
settings: linear system solver = qdldl,
          eps_abs = 1.0e-03, eps_rel = 1.0e-03,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: off, time_limit: off

iter   objective    pri res    dua res    rho        time
   1   0.0000e+00   3.72e+01   3.72e+03   1.00e-01   6.79e-05s
  50   2.3087e+03   3.55e-02   1.35e-04   1.00e-01   1.44e-04s

status:               solved
number of iterat

Row,Generator,Time,Power_Output
,Int64,Int64,Float64
1,1,1,2.91892
2,1,2,2.85405
3,1,3,2.78919
4,1,4,2.72432
5,1,5,4.28108
6,1,6,5.83784
7,1,7,7.39459
8,1,8,8.95135
9,1,9,10.5081
